In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

def find_min_datetime(series):
    # Ensure the series is of datetime type
    series = pd.to_datetime(series)
    
    # Find and return the minimum datetime value
    return series.min()

def find_max_datetime(series):
    # Ensure the series is of datetime type
    series = pd.to_datetime(series)
    
    # Find and return the maximum datetime value
    return series.max()

def get_datetime_range(series):
    # Ensure the series is of datetime type
    series = pd.to_datetime(series)
    
    # Find and return the range of datetime values
    return (series.min(), series.max())

def create_time_range(start, end):
    # Create a range of datetime values
    time_range = {
        "start": datetime.strptime(start, "%Y-%m-%d %H:%M:%S"),
        "end": datetime.strptime(end, "%Y-%m-%d %H:%M:%S")
    }
    
    return time_range

def randomDate(time_range):
    while True:
        try:
            start = time_range["start"]
            end = time_range["end"]
            delta = end - start
            int_delta = delta.days
            random_day = random.randint(0, int_delta)
            result = (start + timedelta(days=random_day)).date().strftime("%Y-%m-%d")
            datetime.strptime(result, "%Y-%m-%d")  # Validate the date
            if result.endswith("02-29"):
                continue  # Avoid 29th February
            return result
        except ValueError:
            continue

def createADateLaterThan(date):
    while True:
        try:
            result = (date + timedelta(days=random.randint(1, 365))).date().strftime("%Y-%m-%d")
            datetime.strptime(result, "%Y-%m-%d")  # Validate the date
            if result.endswith("02-29"):
                continue  # Avoid 29th February
            return result
        except ValueError:
            continue
        
def isDateBetween(date, start, end):
    return datetime.strptime(start,"%Y-%m-%d") <= datetime.strptime(date,"%Y-%m-%d") \
                                            <= datetime.strptime(end,"%Y-%m-%d") 

In [2]:
CHONMON = pd.read_csv('ChonMon.csv')
PHIEUDATMON = pd.read_csv('PhieuDatMon.csv')
NHANVIEN = pd.read_csv('NhanVien.csv')
CHINHANH = pd.read_csv('ChiNhanh.csv')
DOICN = pd.read_csv('DoiCN.csv')

In [28]:
def findValidBranch(NgayLap, MaNV):
    history = DOICN[DOICN['MaNV'] == MaNV]
    NhanVien = NHANVIEN[NHANVIEN['MaNV'] == MaNV]
    for index, row in history.iterrows():
        if isDateBetween(NgayLap, row['NgayBatDau'], row['NgayKetThuc']):
            return row['MaCN']
    return NhanVien['CN_HienTai'].values[0]

In [29]:
for index, row in PHIEUDATMON.iterrows():
    NgayLap = datetime.strptime(row['NgayLap'], "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d")
    MaNV = row['MaNV']
    PHIEUDATMON.at[index, 'MaCN'] = findValidBranch(NgayLap, MaNV)

# Save the updated DataFrame to a CSV file
PHIEUDATMON.to_csv('PhieuDatMon.csv', index=False)

In [3]:
PHIEUDATMON = pd.read_csv('PhieuDatMon.csv')
PHIEUDATMON['MaCN'] = PHIEUDATMON['MaCN'].astype(int)
BAN = pd.read_csv('Ban.csv')
BAN['MaCN'] = BAN['MaCN'].astype(int)

In [4]:
MaBan = []
for index, row in PHIEUDATMON.iterrows():
    MaCN = row['MaCN']
    BAN_CN = BAN[BAN['MaCN'] == MaCN]
    MaBan.append(random.choice(BAN_CN['MaBan'].values))


In [ ]:
PHIEUDATMON['MaBan'] = MaBan
PHIEUDATMON = PHIEUDATMON.drop(columns=['sql_command'], inplace=True)
PHIEUDATMON.to_csv('PhieuDatMon.csv', index=False)

In [12]:
# Generate insert SQL commands
def create_insert_command(index_row):
    index, row = index_row
    values = "', '".join([str(value) for value in row.values])
    return f"INSERT INTO PHIEUDATMON ({', '.join(PHIEUDATMON.columns)}) VALUES ('{values}');"

insert_commands = list(map(create_insert_command, PHIEUDATMON.iterrows()))

# Write the commands to an SQL file
with open('../src/backend/database/Insert_PhieuDatMon.sql', 'w') as file:
    for command in insert_commands:
        file.write(command + '\n')

In [2]:
import pandas as pd
import random
import numpy as np

HOADON = pd.read_csv('HoaDon.csv')

PHIEUDANHGIA = pd.DataFrame()

PHIEUDANHGIA['MaPhieuDG'] = np.arange(1, 100001)
PHIEUDANHGIA['MaHD'] = np.random.choice(HOADON['MaHD'], 100000, replace=False)
PHIEUDANHGIA['DiemPhucVu'] = np.random.randint(2, 6, 100000)
PHIEUDANHGIA['DiemViTri'] = np.random.randint(2, 6, 100000)
PHIEUDANHGIA['DiemMonAn'] = np.random.randint(2, 6, 100000)
PHIEUDANHGIA['DiemKhongGian'] = np.random.randint(2, 6, 100000)
PHIEUDANHGIA['DiemGiaCa'] = np.random.randint(2, 6, 100000)

PHIEUDANHGIA.to_csv('PhieuDanhGia.csv', index=False)

In [3]:
x = np.random.choice(HOADON['MaHD'], 100000, replace=False)
print(len(x))

100000
100000


In [ ]:
commands = []
for i in range(0, len(PHIEUDANHGIA), 1000):
    chunk = PHIEUDANHGIA.iloc[i:i+1000]
    sql_command = "INSERT INTO PHIEUDANHGIA (MaPhieuDG, MaHD, DiemPhucVu, DiemViTri, DiemMonAn, DiemKhongGian, DiemGiaCa) VALUES "
    values_list = map(lambda row: f"({', '.join([str(value) for value in row[1].values])})", chunk.iterrows())
    sql_command += ', '.join(values_list) + "; \nGO\n"
    commands.append(sql_command)

# Write the commands to an SQL file
with open('../src/backend/database/Insert_PhieuDanhGia.sql', 'w') as file:
    for command in commands:
        file.write(command + '\n')


In [2]:
import pandas as pd
import random
import numpy as np

PHUCVU = pd.read_csv('PhucVu.csv')
CHINHANH = pd.read_csv('ChiNhanh.csv')

In [7]:

x = PHUCVU.join(CHINHANH.set_index('MaCN'), on='MaCN')
x[['MaCN', 'MaMon', 'MaKV']].sort_values(by=['MaCN', 'MaMon', 'MaKV']).to_csv('PhucVu.csv', index=False)

In [21]:
ThucDon = x[['MaMon', 'MaKV']]

ThucDon = ThucDon.drop_duplicates()
ThucDon.to_csv('ThucDon.csv', index=False)

In [22]:
sql_command = "INSERT INTO THUCDON (MaMon, MaKV) VALUES "
values_list = map(lambda row: f"({', '.join([str(value) for value in row[1].values])})", ThucDon.iterrows())

with open('../src/backend/database/Insert_ThucDon.sql', 'w') as file:
    file.write(sql_command + ', '.join(values_list) + "; \nGO\n")

In [3]:
GhiChu = ["Bàn gần cửa sổ.",
"Bàn ngoài trời.",
"Bàn riêng tư.",
"Bàn gần quầy bar.",
"Bàn trong phòng VIP.",
"Bàn gần cửa ra vào.",
"Bàn gần khu vực trẻ em.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi.",
"Bàn gần khu vực tempura.",
"Bàn gần khu vực lẩu.",
"Bàn gần khu vực nướng.",
"Bàn gần khu vực sushi.",
"Bàn gần khu vực sashimi."]

In [4]:
import pandas as pd
import numpy as np


DATBAN = pd.read_csv('DatBan.csv')
DATBAN['MADATBAN'] = np.arange(1, len(DATBAN) + 1)
DATBAN['GHICHU'] = np.random.choice(GhiChu, len(DATBAN))
DATBAN['SOLUONG'] = np.random.randint(1, 11, len(DATBAN))

In [5]:
DATBAN = DATBAN[['MADATBAN', 'HOTEN', 'SDT', 'NGAYGIODAT', 'SOLUONG', 'CHINHANH', 'GHICHU', 'MAHD']]

In [6]:
sql_command = "INSERT INTO DATBAN (MaDatBan, HoTen, SDT, NgayGioDat, SoLuong, ChiNhanh, GhiChu, MaHD) VALUES "

values_list = []

for index, row in DATBAN.iterrows():
    value = f"({row['MADATBAN']}, N'{row['HOTEN']}', '{row['SDT']}', '{row['NGAYGIODAT']}', {row['SOLUONG']}, {row['CHINHANH']}, N'{row['GHICHU']}', {row['MAHD']})"
    values_list.append(value)

with open('../src/backend/database/Insert_DatBan.sql', 'w') as file:
    file.write(sql_command + ',\n'.join(values_list) + "; \nGO\n")